In [1]:
# Import required libraries

import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import L2
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

2024-04-21 11:28:21.570091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Data preprocessing

def get_data_preprocessed(dataframe):
    
    # Drop columns which doesn't contribute much to the survival decision
    dataframe = dataframe.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
    
    # One hot encoding of categorical columns
    dataframe = pd.get_dummies(dataframe, columns=['Sex', 'Embarked'], drop_first=True)
    
    # Fill missing values
    dataframe['Age'].fillna(dataframe['Age'].mean(), inplace=True)
    
    
    return dataframe

In [3]:
# Split train.csv in train, test dataset

df = pd.read_csv('data.csv')
X = get_data_preprocessed(df)
X = X.drop('Survived', axis=1).astype(float)
y = df[['Survived']].astype(float)
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2)

In [5]:
# Prepare model using neural network

def prepare_model(X_train, y_train):
    model = Sequential([
        tf.keras.Input(shape=(8,)),
        Dense(units=32, activation='relu'),
        Dense(units=16, activation='relu'),
        Dense(units=1, activation='sigmoid', kernel_regularizer=L2(0.01))
    ], name='my_model'
    )

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(0.001),
    )

    model.fit(
        X_train, y_train,
        epochs=100,
        verbose=1
    )
    return model

model = prepare_model(X_train, y_train)

Epoch 1/100
23/23 [==============================] - 1s 4ms/step - loss: 1.6146
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.7046
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6561
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 0.6464
Epoch 5/100
23/23 [==============================] - 0s 8ms/step - loss: 0.6252
Epoch 6/100
23/23 [==============================] - 0s 14ms/step - loss: 0.6145
Epoch 7/100
23/23 [==============================] - 0s 8ms/step - loss: 0.6120
Epoch 8/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6055
Epoch 9/100
23/23 [==============================] - 0s 4ms/step - loss: 0.5917
Epoch 10/100
23/23 [==============================] - 0s 5ms/step - loss: 0.6016
Epoch 11/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5857
Epoch 12/100
23/23 [==============================] - 0s 9ms/step - loss: 0.5802
Epoch 13/100
23/23 [================

In [6]:
# Convert the probability predictions to binary predictions based on a threshold (0.5)
y_pred_prob = model.predict(X_cv)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Convert the true labels to int
y_true = y_cv.values.astype(int)

6/6 [==============================] - 0s 9ms/step


In [7]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Print a classification report for more detailed metrics
report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(report)

Confusion Matrix:
[[103  18]
 [ 16  42]]

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       121
           1       0.70      0.72      0.71        58

    accuracy                           0.81       179
   macro avg       0.78      0.79      0.79       179
weighted avg       0.81      0.81      0.81       179

